In [ ]:
import os
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from operator import itemgetter
import pygraphviz

In [ ]:
os.getcwd()

# Load the data

In [ ]:
# import data
df = pd.read_excel('European_electricity_network_vFinal.xlsx', sheet_name='2014')
df.head()

In [ ]:
# drop unnecessary columns
df_elec = df.drop(['Origin_Country'], axis=1)

# Convert the wide table to long table
Country_list = list(df_elec.columns)
df_elec = df_elec.melt(id_vars=['Country_Code'], value_vars=(Country_list[1:]), var_name='Destination', value_name='Electricity_Flow')
df_elec.rename(columns={'Country_Code':'Origin'}, inplace=True)

# Sort the table based on origin country's name
df_elec.sort_values('Origin', inplace=True, ascending=True)

# Delete NAs (when Origin=Destination)
df_elec.dropna(axis=0, inplace=True) 

# Delete zero-value rows
df_elec = df_elec[df_elec.Electricity_Flow != 0]
df_elec.reset_index(drop=True, inplace=True)

df_elec.head()

In [ ]:
len(df_elec)

# Preparation before creating network

## Create nodes list

In [ ]:
node = df_elec['Origin'].unique()
node_list = list(node)
node_list[:5]

## Calculate edge weights

In [ ]:
ebunch = list(df_elec.to_records(index=False))

In [ ]:
# use "Electricity_Flow" as weights
weights_list = list(df_elec['Electricity_Flow'])

In [ ]:
edge_width_list = [None]*len(weights_list)

# use max(weight) to normalize the size of edges
max_weights = max(weights_list)

for i, weight in enumerate(weights_list):
    edge_width_list[i] = weights_list[i]/max_weights*5

# create an array of edge widths to be assigned to network
edge_width = np.asarray(edge_width_list)

## Calculate node size: total export of each country

In [ ]:
# calculate total export for each country
df_total_export = df_elec.groupby(['Origin'], as_index=False).sum()
df_total_export.columns = ['Country', 'Export']
df_total_export.head()

In [ ]:
# convert export dataframe to np array, to be assigned to node_size
total_export = np.asarray(df_total_export['Export'])

In [ ]:
# check top exporting countries
df_total_export_sorted = df_total_export.sort_values(by='Export', ascending=False)
df_total_export_sorted.head()

# Create networks
## Directed network

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(node_list)
G.add_weighted_edges_from(ebunch_to_add=ebunch, weight='weight')

In [ ]:
pos = nx.spring_layout(G, k=8, scale=6, seed=15)

node_size = total_export/20
node_color = [float(G.degree(i)) for i in G]

plt.figure(figsize=(15,13))

nodes = nx.draw_networkx_nodes(G, pos, node_size=node_size, node_color=node_color, alpha=0.5)
edges = nx.draw_networkx_edges(G, pos, width=edge_width, edge_color='skyblue', 
                       arrowstyle='-|>', arrowsize=14,
                       connectionstyle='arc3,rad=0.1')
nx.draw_networkx_labels(G, pos, font_size=12)

plt.title('European electricity export/import network (2014)', fontsize=15)
plt.text(1.5,-4,'Node color is degree centrality; node size is total export', fontsize=12)
cbar = plt.colorbar(mappable=nodes, cax=None, ax=None, fraction=0.015, pad=0.04)

plt.axis('off')
plt.show()

# plt.margins(0,0)
# plt.savefig("European electricity export/import network (2014).png", dpi=1000)

In [ ]:
list(G.degree())

In [ ]:
nx.betweenness_centrality(G)